# 举例1：大模型分析工具的调用

In [1]:
# 1、获取大模型
#导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.tools import StructuredTool
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数:convert_to_openai_function()
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions=functions,
)

print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 76, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CDMNkGJTet5BejWsNTnkE9fpJAtxv', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None} id='run--587f6b3c-f8a6-4bde-a26e-41d67e1a5b1c-0' usage_metadata={'input_tokens': 76, 'output_tokens': 27, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


作为对比：


In [2]:
# 获取消息列表
messages = [HumanMessage(content="查询一下明天北京的天气")]

# 调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions=functions,
)

print(response)

content='抱歉，我无法提供实时天气信息。建议您查看天气预报网站或使用天气应用程序获取最新的天气信息。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 76, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CDMPcUVaacw6eKtuHKMVwdKmLkZ9c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--ccf690d0-4b10-4808-91ff-24abfea18ceb-0' usage_metadata={'input_tokens': 76, 'output_tokens': 29, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


通过上面两个测试发现，得到的AIMessage的核心属性如下：

1、如果分析出需要调用对应的工具：

content：信息为空。因为大模型要调用工具，所以就不会直接返回信息给用户

additional_kwargs：包含function_call字段，指明具体函数调用的参数和函数名。比如：

additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None}

2、如果分析出不需要调用对应的工具：

content：信息不为空。

additional_kwargs：不包含function_call字段








# 举例2：如何调用具体大模型分析出来的工具

说明：

1、大模型与Agent的核心区别：是否涉及到工具的调用

2、针对于大模型：仅能分析出要调用的工具，但是此工具（或函数）不能真正的执行

   针对于Agent:除了分析出要调用的工具之外，还可以执行具体的工具（或函数）

In [8]:
# 1、获取大模型
#导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数:convert_to_openai_function()
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content="将当前目录下的文件a.txt移动到C:\\Users\\shkst\\Desktop")]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions=functions,
)

# print(response)

步骤1：分析下要调用哪个工具或函数

In [11]:
import json

if "function_call" in response.additional_kwargs:
    tool_name = response.additional_kwargs["function_call"]["name"]
    tool_args = json.loads(response.additional_kwargs["function_call"]["arguments"])
    print(f"调用工具：{tool_name} \n 参数：{tool_args}")

else:
    print(f"模型回复：{response.content}")

调用工具：move_file 
 参数：{'source_path': 'a.txt', 'destination_path': 'C:\\Users\\shkst\\Desktop\\a.txt'}


步骤2：调用对应的工具

In [12]:
if "move_file" in response.additional_kwargs["function_call"]["name"]:
    tool = MoveFileTool()
    result = tool.run(tool_args)  #调用工具
    print("工具执行的结果", result)

工具执行的结果 File moved successfully from a.txt to C:\Users\shkst\Desktop\a.txt.
